In [1]:
from scipy.stats import chi2, binom, uniform, expon, norm
from random import gauss, gammavariate
from math import sqrt, erf

In [2]:
def T(k, n, p_i, N):
    """
    Estadístico Chi Cuadrado.

    Args:
        k (int): cantidad de valores que puede tomar la variable aleatoria
        n (int): tamaño muestral
        p_i (list[float]): probabilidades teóricas para cada valor que puede
        tomar la distribución de la hipótesis nula
        N (list[int]): frecuencias observadas

    Returns:
        t (float): estadístico Chi Cuadrad observado a partir de la muestra
    """
    t = 0
    for i in range(k):
        t += (N[i] - n*p_i[i])**2 / (n*p_i[i])
    return t

def D(n, X, F):
    """
    Estadístico de Kolmogorov-Smirnov.

    Args:
        n (int): tamaño muestral
        X (list[float]): muestra obtenida
        F (function): función de distribución acumulada de la distribución planteada
        en la hipótesis nula

    Returns:
        d (float): estadístico de Kolmogorov-Smirnov observado a partir de la muestra
    """
    d = 0
    for j in range(n):
        Fe_F = (j+1)/n - F(X[j])
        F_Fe = F(X[j]) - j/n
        d = max(d, Fe_F, F_Fe)
    return d

def aprox_p_valor_con_uniformes(nSim, n, d):
    """
    Aproximación del p-valor de una distribución continua con simulaciones.

    Args:
        nSim (int): cantidad de simulaciones a realizar
        n (int): tamaño de cada muestra a generar
        d (float): estadístico de Kolmogorov-Smirnov obtenido a partir de la muestra
        original

    Returns:
        p_valor/nSim (float): aproximación del p-valor en nSim simulaciones
    """
    p_valor = 0
    nSim = 1000
    for _ in range(nSim):
        uniformes = uniform.rvs(size=n)
        uniformes.sort()
        d_j = D(n, uniformes, uniform.cdf)
        if d_j >= d:
            p_valor += 1
    return p_valor/nSim

def frecuencias_observadas_discreta(n, k, p_i):
    """
    Algoritmo para simular las frecuencias observadas cuando se trata de una
    variable aleatoria discreta.

    Args:
        n (int): tamaño muestral
        k (int): cantidad de valores que puede tomar la variable aleatoria
        p_i (list[float]): probabilidades teóricas para cada valor que puede
        tomar la distribución de la hipótesis nula

    Returns:
        NN (list[int]): frecuencias observadas simuladas
    """
    N = [0] * k
    for i in range(k):
        p = p_i[i] / (1 - sum(p_i[:i]))
        N[i] = binom.rvs(n, p)
        n -= N[i]
    return N

# Algoritmo para simular N_0, N_1,..., N_k-1 cuando se trata de una v.a. continua
def frecuenciasObservadasContinua(y_i, X_i):
    N = [0] * (len(y_i) + 1)
    X_i.sort()
    i = 0
    j = 0
    while i < len(X_i):
        if X_i[i] <= y_i[j]:
            N[j] += 1
            i += 1
        else:
            j += 1
            if j == len(y_i):   # Esto ocurre solo si hay algún valor que sea mayor al último y_i
                break
    N[-1] = len(X_i) - sum(N)
    return N

## Ejercicio 1
De acuerdo con la teoría genética de Mendel, cierta planta de guisantes debe producir flores blancas, rosas o rojas con probabilidad 1/4, 1/2 y 1/4, respectivamente. Para verificar experimentalmente la teoría, se estudió una muestra de 564 guisantes, donde se encontró que 141 produjeron flores blancas, 291 flores rosas y 132 flores rojas. Aproximar el p-valor de esta muestra:  

In [7]:
# Tamaño muestral
n = 564

# Valores de la muestra
k = 3

# Frecuencias observadas
N_i = [141, 291, 132]
p_i = [1/4, 1/2, 1/4]

# Estadístico
t = T(k, n, p_i, N_i)
print(f'Valor observado del estadístico T: {t}')

Valor observado del estadístico T: 0.8617021276595745


&emsp; **a)** utilizando la prueba de Pearson con aproximación chi-cuadrada  

In [8]:
p_valor = 1 - chi2.cdf(t, k-1)
print(f'p-valor con test Chi-cuadrado: {p_valor}')

p-valor con test Chi-cuadrado: 0.6499557054800363


&emsp; **b)** realizando una simulación.

In [9]:
# Para la simulación, genero las frecuencias observadas
nSim = 1000
p_valor = 0
for i in range(nSim):
    N_i = frecuencias_observadas_discreta(n, k, p_i)
    t_i = T(k, n, p_i, N_i)
    if t_i >= t:
        p_valor += 1
print(f'p-valor con {nSim} simulaciones: {p_valor / nSim}')

p-valor con 1000 simulaciones: 0.611


## Ejercicio 2
Para verificar que cierto dado no estaba trucado, se registraron 1000 lanzamientos, resultando que el número de veces que el dado arrojó el valor i (i = 1,2,3,4,5,6) fue, respectivamente, 158, 172, 164, 181, 160, 165. Aproximar el p−valor de la prueba: “el dado es honesto”

NOTAR: la prueba de "el dado es honesto" se traduce como "los valores de la muestra provienen de una distribución uniforme discreta U ~ {1,...6}", donde en este caso $p_i = 1/6$ para i = 1,...,6

&emsp; **a)** Utilizando la prueba de Pearson con aproximación chi-cuadrada,


In [ ]:
# Tamaño muestral
n = 1000

# Valores de la muestra
k = 6

# Frecuencias observadas
N_i = [158, 172, 164, 181, 160, 165]
p_i = [1/k] * 6

# Estadístico
t = T(k, n, p_i, N_i)
print(f'Valor observado del estadístico T: {t}')

In [ ]:
p_valor = 1 - chi2.cdf(t, k-1)
print(f'p-valor con test Chi-cuadrado: {p_valor}')

&emsp; **b)** Realizando una simulación.

In [ ]:
nSim = 1000
p_valor = 0
for i in range(nSim):
    N_i = frecuencias_observadas_discreta(n, k, p_i)
    t_i = T(k, n, p_i, N_i)
    if t_i >= t:
        p_valor += 1
print(f'p-valor con {nSim} simulaciones: {p_valor / nSim}')

## Ejercicio 3
Calcular una aproximación del p−valor de la hipótesis: “Los siguientes 10 números son aleatorios”:  
&emsp; 0.12, 0.18, 0.06, 0.33, 0.72, 0.83, 0.36, 0.27, 0.77, 0.74.

Usando **test de Kolmogorov - Smirnov**

In [ ]:
X = [0.12, 0.18, 0.06, 0.33, 0.72, 0.83, 0.36, 0.27, 0.77, 0.74]
n = len(X)

# 1) Ordenamos los datos de menor a mayor
X.sort()

# 2) Calculamos el estadístico
d = D(n, X, uniform.cdf)
print(f'Valor observado del estadístico D: {d}')

# 3) Aproximamos el p-valor con simulaciones usando teorema
p_valor_sim = aprox_p_valor_con_uniformes(1000, n, d)
print(f'p-valor con {nSim} simulaciones: {p_valor_sim}')


## Ejercicio 4
Calcular una aproximación del p−valor de la hipótesis: “Los siguientes 13 valores provienen de una distribución exponencial con media 50.0”:  
&emsp; 86.0 , 133.0 , 75.0 , 22.0 , 11.0 , 144.0 , 78.0 , 122.0 , 8.0 , 146.0 , 33.0 , 41.0 , 99.0 .

Usando **test de Kolmogorov - Smirnov**

In [ ]:
X = [86.0 , 133.0 , 75.0 , 22.0 , 11.0 , 144.0 , 78.0 , 122.0 , 8.0 , 146.0 , 33.0 , 41.0 , 99.0]
n = len(X)

# 1) Ordenamos los datos de menor a mayor
X.sort()

# 2) Calculamos el estadístico
lamda = 1/50
d = D(n, X, lambda x: expon.cdf(x, scale=1/lamda))
print(f'Valor observado del estadístico D: {d}')

# 3) Aproximamos el p-valor con simulaciones usando teorema
p_valor_sim = aprox_p_valor_con_uniformes(1000, n, d)
print(f'p-valor con {nSim} simulaciones: {p_valor_sim}')

## Ejercicio 5
Calcular una aproximación del p−valor de la prueba de que los siguientes datos corresponden a una distribución binomial con parámetros (n = 8, p), donde p no se conoce:  
&emsp; 6, 7, 3, 4, 7, 3, 7, 2, 6, 3, 7, 8, 2, 1, 3, 5, 8, 7.

Como en este caso no tenemos p especificado, tenemos que usar su estimador. Para ello, notemos que si X ~ B(n,p), entonces E[X] = np, y además tenemos que si tenemos una muestra $X_1,...,X_m$ con $X_i \sim B(n,p)$, entonces el estimador de E[X] es la media muestral.  
En este caso, bajo la hipótesis nula, tenemos que los datos provienen de una binomial con parámetros (n=8, p). Por lo tanto, para estimar E[X] y así poder obtener una estimación de p, calculamos la media muestral de los datos.

In [3]:
X = [6, 7, 3, 4, 7, 3, 7, 2, 6, 3, 7, 8, 2, 1, 3, 5, 8, 7]
tam_muestra = len(X)
k = 9
n = 8

# Estimamos p a partir de la muestra
media_muestral = sum(X) / tam_muestra
_p = media_muestral / n

# Estimamos las probabilidades usando la p estimada
p_i = [binom.pmf(x, n=n, p=_p) for x in range(k)]

# Frecuencias observadas
N = [0] * k
for i in range(tam_muestra):
    N[X[i]] += 1

# Estadístico
t = T(k, tam_muestra, p_i, N)
print(f'Valor observado del estadístico T: {t}')

Valor observado del estadístico T: 31.49933093415532


Con **test Chi-cuadrado**

In [4]:
p_valor = 1 - chi2.cdf(t, k-1-1)    # Hay que restar un grado de libertad por cada parámetro estimado
print(f'p-valor con test Chi-cuadrado: {p_valor:.10f}')

p-valor con test Chi-cuadrado: 0.0000502799


Con **simulaciones**

In [6]:
nSim = 1000
p_valor = 0

for i in range(nSim):
    # 1) Se simula X1,...,Xn con _p_orig (estimación de p a partir de la muestra original)
    X = binom.rvs(n=8, p=_p, size=tamMuestra)
    
    # 2) Se calculan las frecuencias observadas
    N = [0] * k
    for value in X:
        N[value] += 1
    
    # 3) Se vuelve a estimar p
    mediaMuestral = sum(X) / tamMuestra
    _p_sim = mediaMuestral / 8
    
    # 4) Se vuelven a calcular las probabilidades con el nuevo parámetro estimado
    p_i = [binom.pmf(x, n=8, p=_p_sim) for x in range(k)]
    
    # 5) Se calcula el estadístico
    t_i = T(k, tamMuestra, p_i, N)
    if t_i >= t:
        p_valor += 1

print(f'p-valor con {nSim} simulaciones: {p_valor / nSim}')

p-valor con 1000 simulaciones: 0.01


## Ejercicio 6
Un escribano debe validar un juego en cierto programa de televisión. El mismo consiste en hacer girar una rueda y obtener un premio según el sector de la rueda que coincida con una aguja. Hay 10 premios posibles, y las áreas de la rueda para los distintos premios, numerados del 1 al 10, son
respectivamente:
$$
31\%, 22\%, 12\%, 10\%, 8\%, 6\%, 4\%, 4\%, 2\% \; y \; 1\%.
$$
Los premios con número alto (e.j. un auto 0Km) son mejores que los premios con número bajo (e.j. 2x1 para entradas en el cine). El escribano hace girar la rueda hasta que se cansa, y anota cuántas veces sale cada sector. Los resultados, para los premios del 1 al 10, respectivamente, son:  
$$
188, 138, 87, 65, 48, 32, 30, 34, 13 \; y \; 2.
$$

**NOTAR**: los datos que da en el ejercicio (188, 138, 87, ...) son las frecuencias observadas de la muestra. Es decir, la muestra que se ontuvo es una en donde aparece:  
- 188 veces el número 1
- 138 veces el número 2
- 87 veces el número 3
- ...
- 2 veces el número 10

&emsp; **(a)** Construya una tabla con los datos disponibles

&emsp; **(b)** Diseñe una prueba de hipótesis para determinar si la rueda es justa

&emsp;&emsp; $H_0$: los valores de la muestra provienen de una variable aleatoria X tal que
$$
p_1 = 0.31, \; p_2 = 0.22, \; p_3 = 0.12, \; p_4 = 0.1, \; p_5 = 0.08, \; p_6 = 0.06, \; p_7 = 0.04, \; p_8 = 0.04, \; p_9 = 0.02, \; p_10 = 0.01, \; donde \; p_i = P(X = i)
$$

&emsp; **(c)** Defina el p-valor a partir de la hipótesis nula

Dado:
$$
    T = \sum_{i=1}^{10}\frac{(N_i - np_i)^2}{np_i}
$$
donde
- $N_i\text{ son las frecuencias observadas en la muestra}: N_1 = 188, \; N_2 = 138, \; N_3 = 87, \; N_4 = 65, \; N_5 = 48, \; N_6 = 32, \; N_7 = 30, \; N_8 = 34, \; N_9 = 13, \; N_{10} = 2,$
- $\text{n es el tamaño de la muestra}, $
- $p_i \text{ son las probabilidades teóricas}: p_1 = 0.31, \; p_2 = 0.22, \; p_3 = 0.12, \; p_4 = 0.1, \; p_5 = 0.08, \; p_6 = 0.06, \; p_7 = 0.04, \; p_8 = 0.04, \; p_9 = 0.02, \; p_{10} = 0.01$

el p-valor se lo define como:
$$
p-valor = P(T \ge t)
$$,
donde t es el valor obervado de T a partir de la muestra original.

&emsp; **(d)** Calcule el p-valor bajo la hipótesis de que la rueda es justa, usando la aproximación chi cuadrado

In [ ]:
# Frecuencias observadas
N = [188, 138, 87, 65, 48, 32, 30, 34, 13, 2]

# Probabilidades
p_i = [0.31, 0.22, 0.12, 0.1, 0.08, 0.06, 0.04, 0.04, 0.02, 0.01]

# Tamaño muestral
n = sum(N)

# Valores posibles: del 1 al 10
k = 10

# Estadístico
t = T(k, n, p_i, N)
print(f'Valor observado del estadístico T: {t}')

# p-valor usando la aproximación chi cuadrado
p_valor = 1 - chi2.cdf(t, k-1)
print(f'p-valor con test Chi-cuadrado: {p_valor:.10f}')

&emsp; **(e)** Calcule el p-valor bajo la hipótesis de que la rueda es justa, usando una simulación.

In [ ]:
nSim = 1000
p_valor = 0
for i in range(nSim):
    N_i = frecuencias_observadas_discreta(n, k, p_i)
    t_i = T(k, n, p_i, N_i)
    if t_i >= t:
        p_valor += 1
print(f'p-valor con {nSim} simulaciones: {p_valor / nSim}')

## Ejercicio 7
Generar los valores correspondientes a 30 variables aleatorias exponenciales independientes, cada una con media 1. Luego, en base al  estadístico de prueba de Kolmogorov-Smirnov, aproxime el p−valor de la prueba de que los datos realmente provienen de una distribución exponencial con media 1.

In [ ]:
# H_0: los datos de la muestra provienen de una variable aleatoria exponencial con media 1

# Tamaño muestral
n = 100
X = expon.rvs(scale=1, size=n)

# 1) Ordenamos los datos de menor a mayor
X.sort()

# 2) Calculamos el estadístico
d = D(n, X, lambda x: expon.cdf(x, scale=1))
print(f'Valor observado del estadístico: {d}')

# 3) Aproximamos el p-valor con simulaciones usando teorema
p_valor_sim = aprox_p_valor_con_uniformes(1000, n, d)
print(f'p-valor con {nSim} simulaciones: {p_valor_sim}')

## Ejercicio 8
Se sortean elementos de un conjunto de datos que tiene una distribución t-student de 11 grados de libertad. El investigador, que no conoce la forma verdadera de la distribución, asume que la misma es normal.


Analice la validez de esta suposición para muestras de tamaños 10, 20, 100 y 1000 elementos. Para ello realice simulaciones numéricas e implemente el test de Kolmogorov-Smirnov a los datos simulados, asumiendo una distribución N(0,1). Presente los resultados en una tabla que contenga el número de elementos de la simulación, el valor del estadístico D y el p−valor.


Ayuda: Función de probabilidad normal: Para obtener la función de probabilidad normal, se puede usar la función `math.erf`. Por ejemplo, la cantidad `math.erf(x/math.sqrt(2.))/2.+0.5` equivale a
$$
\int_{-\infty}^{x} N(0,1)(t)dt
$$
Ayuda: Generación de números aleatorios con una distribución t-student: Utilice el siguiente código para generar números aleatorios que siguen una distribución T-student:
``` python
import math
import random

def rt(df): # df grados de libertad
    x = random.gauss(0.0, 1.0)
    y = 2.0*random.gammavariate(0.5*df, 2.0)
    return x / (math.sqrt(y/df))
```

In [ ]:
def rt(df): # df grados de libertad
    x = gauss(0.0, 1.0)
    y = 2.0 * gammavariate(0.5*df, 2.0)
    return x / sqrt(y/df)

df = 11

In [ ]:
# Tamaños muestrales
tams = [10, 20, 100, 1000]
for n in tams:
    print(f'Con una muestra de tamaño {n}:')
    X = [rt(df) for _ in range(n)]

    # 1) Ordenamos los datos de menor a mayor
    X.sort()

    # 2) Calculamos el estadístico
    d = D(n, X, lambda x: erf(x/sqrt(2.))/2. + 0.5)
    print(f'\tValor observado del estadístico: {d}')

    # 3) Aproximamos el p-valor con simulaciones usando teorema: P_F(D >= d) no depende de F
    p_valor_sim = aprox_p_valor_con_uniformes(1000, n, d)
    print(f'\tp-valor con {nSim} simulaciones: {p_valor_sim}')

## Ejercicio 9
En un estudio de vibraciones, una muestra aleatoria de 15 componentes del avión fueron sometidos a fuertes vibraciones hasta que se evidenciaron fallas estructurales. Los datos proporcionados son los minutos transcurridos hasta que se evidenciaron dichas fallas.

&emsp; 1.6 10.3 3.5 13.5 18.4 7.7 24.3 10.7 8.4 4.9 7.9 12 16.2 6.8 14.7

Pruebe la hipótesis nula de que estas observaciones pueden ser consideradas como una muestra de la
distribución exponencial.

In [ ]:
# Muestra
X = [1.6, 10.3, 3.5, 13.5, 18.4, 7.7, 24.3, 10.7, 8.4, 4.9, 7.9, 12, 16.2, 6.8, 14.7]

# Tamaño muestral
n = len(X)

# 1) Ordenamos los datos de menor a mayor
X.sort()

# 2) Estimamos los parámetros desconocidos (en este caso lambda)
# Sabemos que si X ~ Exp(lambda), entonces E[X] = 1 / lambda. Por lo tanto lambda = 1 / E[X]
# Entonces, para estimar lambda, estimamos E[X] con la media muestral y hago 1 / MediaMuestral
media_muestral = sum(X) / n
_lamda = 1 / media_muestral

# 3) Calculamos el estadístico
d = D(n, X, lambda x: expon.cdf(x, scale=1/_lamda))
print(f'Valor observado del estadístico: {d}')

# 4) Estimamos el p-valor con simulaciones
# NOTA: como los parámetros son desconocidos, no podemos usar muestras de uniformes
# en la simulación
nSim = 1000
p_valor = 0
for _ in range(nSim):
    X = expon.rvs(scale=1/_lamda, size=n)
    X.sort()

    # Se vuelven a estimar los parámetros
    media_muestral = sum(X) / n
    _lamda_sim = 1 / media_muestral

    # Calculamos el estadístico con los nuevos parámetros
    d_sim = D(n, X, lambda x: expon.cdf(x, scale=1/_lamda_sim))
    if d_sim >= d:
        p_valor += 1
print(f'p-valor con {nSim} simulaciones: {p_valor/nSim}\n')

## Ejercicio 10
Decidir si los siguientes datos corresponden a una distribución Normal:

&emsp; 91.9 97.8 111.4 122.3 105.4 95.0 103.8 99.6 96.6 119.3 104.8 101.7

Calcular una aproximación del p−valor.

In [ ]:
# Muestra
X = [91.9, 97.8, 111.4, 122.3, 105.4, 95.0, 103.8, 99.6, 96.6, 119.3, 104.8, 101.7]

# Tamaño muestral
n = len(X)

# 1) Ordenamos los datos de menor a mayor
X.sort()

# 2) Estimamos los parámetros desconocidos (en este caso lambda)
# Sabemos que si X ~ N(mu,sigma), entonces E[X] = mu y VAR[X] = sigma^2.
# Entonces, para estimar mu y sigma, usamos la media muestral y la varianza muestral.
media_muestral = sum(X) / n
_mu = media_muestral

Scuad = 0
for i in range(n):
    Scuad += (X[i] - _mu)**2
Scuad /= (n-1)
_sigma = sqrt(Scuad)

# 3) Calculamos el estadístico
d = D(n, X, lambda x: norm.cdf(x, loc=_mu, scale=_sigma))
print(f'Valor observado del estadístico: {d}')

# 4) Estimamos el p-valor con simulaciones
# NOTA: como los parámetros son desconocidos, no podemos usar muestras de uniformes
# en la simulación
nSim = 1000
p_valor = 0
for _ in range(nSim):
    X = norm.rvs(loc=_mu, scale=_sigma, size=n)
    X.sort()

    # Se vuelven a estimar los parámetros
    media_muestral = sum(X) / n
    _mu_sim = media_muestral

    Scuad = 0
    for i in range(n):
        Scuad += (X[i] - _mu)**2
    Scuad /= (n-1)
    _sigma_sim = sqrt(Scuad)

    # Calculamos el estadístico con los nuevos parámetros
    d_sim = D(n, X, lambda x: norm.cdf(x, loc=_mu_sim, scale=_sigma_sim))
    if d_sim >= d:
        p_valor += 1
print(f'p-valor con {nSim} simulaciones: {p_valor/nSim}\n')